In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from LSTM import *
import pandas as pd
import numpy as np
import torch.utils.data as utils
import time
import os
import pickle

import pdb

# For the notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

torch.manual_seed(1)

In [2]:
gt_test_coord_1 = pickle.load( open( "../import_dataset_3/test/gt_test_coord_1.pkl", "rb" ) )
in_test_coord_1 = pickle.load( open( "../import_dataset_3/test/in_test_coord_1.pkl", "rb" ) )
inputs_test_1 = pickle.load( open( "../import_dataset_3/test/inputs_test_1.pkl", "rb" ) )

gt_test_coord_2 = pickle.load( open( "../import_dataset_3/test/gt_test_coord_2.pkl", "rb" ) )
in_test_coord_2 = pickle.load( open( "../import_dataset_3/test/in_test_coord_2.pkl", "rb" ) )
inputs_test_2 = pickle.load( open( "../import_dataset_3/test/inputs_test_2.pkl", "rb" ) )

gt_test_coord_3 = pickle.load( open( "../import_dataset_3/test/gt_test_coord_3.pkl", "rb" ) )
in_test_coord_3 = pickle.load( open( "../import_dataset_3/test/in_test_coord_3.pkl", "rb" ) )
inputs_test_3 = pickle.load( open( "../import_dataset_3/test/inputs_test_3.pkl", "rb" ) )

inputs_test_1 = inputs_test_1[:,:,2:]
inputs_test_2 = inputs_test_2[:,:,2:]
inputs_test_3 = inputs_test_3[:,:,2:]

inputs_test = np.concatenate((inputs_test_1,inputs_test_2,inputs_test_3),axis=1)
in_test_coord = np.concatenate((in_test_coord_1,in_test_coord_2,in_test_coord_3),axis=1)
gt_test_coord = np.concatenate((gt_test_coord_1,gt_test_coord_2,gt_test_coord_3),axis=1)

inputs_test = torch.from_numpy(inputs_test).float()

inputs_test_1 = torch.from_numpy(inputs_test_1).float()

inputs_test_2 = torch.from_numpy(inputs_test_2).float()

inputs_test_3 = torch.from_numpy(inputs_test_3).float()

inputs_test_1.shape, inputs_test_2.shape, inputs_test_3.shape,inputs_test.shape



(torch.Size([10, 251, 242]),
 torch.Size([10, 1007, 242]),
 torch.Size([10, 460, 242]),
 torch.Size([10, 1718, 242]))

##### Full LSTM

In [4]:
full_lstm = LSTM(input_size = inputs_test_1.shape[2], output_size = 2,
                 num_layers=2, hidden_size=128)
full_lstm.load_state_dict(torch.load('../model_rodolphe/model_results/speed_ND_2.pt'))


In [5]:
output_test = full_lstm.predict(inputs_test)
output_test = output_test.data.numpy()

output_test_1 = full_lstm.predict(inputs_test_1)
output_test_1 = output_test_1.data.numpy()

output_test_2 = full_lstm.predict(inputs_test_2)
output_test_2 = output_test_2.data.numpy()

output_test_3 = full_lstm.predict(inputs_test_3)
output_test_3 = output_test_3.data.numpy()
output_test_1.shape

(10, 251, 2)

In [11]:
output_coord_1=np.zeros([11,inputs_test_1.shape[1],2])
for j in range(11):
    for i in range(inputs_test_1.shape[1]):
        if j==0:
            output_coord_1[j,i,0:2] = in_test_coord_1[9,i,0:2]
        else:
            output_coord_1[j,i,0:2] = output_test_1[j-1,i,:2]*0.4 + output_coord_1[j-1,i,0:2]


In [12]:
dist = np.zeros(inputs_test_1.shape[1])
for i in range(inputs_test_1.shape[1]):
    dist[i] = np.sqrt(sum((gt_test_coord_1[10,i,:]-output_coord_1[10,i,:])**2))
    
final_coord_error = np.mean(dist)
print('The final distance between the ground trought and the predicted coordinates is :',
      final_coord_error.round(3))

avr = np.zeros((inputs_test_1.shape[1],11))
for i in range(inputs_test_1.shape[1]):
    for j in range(11):
        avr[i,j] = np.sqrt(sum((gt_test_coord_1[j,i,:]-output_coord_1[j,i,:])**2))

average = np.mean(np.mean(avr,1))
print('The average error between the ground trought and the predicted coordinates is :',
      average.round(3))        

The final distance between the ground trought and the predicted coordinates is : 1.412
The average error between the ground trought and the predicted coordinates is : 0.726


In [13]:
output_coord_2=np.zeros([11,inputs_test_2.shape[1],2])
for j in range(11):
    for i in range(inputs_test_2.shape[1]):
        if j==0:
            output_coord_2[j,i,0:2] = in_test_coord_2[9,i,0:2]
        else:
            output_coord_2[j,i,0:2] = output_test_2[j-1,i,:2]*0.4 + output_coord_2[j-1,i,0:2]
        

dist = np.zeros(inputs_test_2.shape[1])
for i in range(inputs_test_2.shape[1]):
    dist[i] = np.sqrt(sum((gt_test_coord_2[10,i,:]-output_coord_2[10,i,:])**2))

final_coord_error = np.mean(dist)
print('The final distance between the ground truth and the predicted coordinates is :',final_coord_error.round(3))

avr = np.zeros((inputs_test_2.shape[1],11))
for i in range(inputs_test_2.shape[1]):
    for j in range(11):
        avr[i,j] = np.sqrt(sum((gt_test_coord_2[j,i,:]-output_coord_2[j,i,:])**2))

average = np.mean(np.mean(avr,1))
print('The average error between the ground truth and the predicted coordinates is :',average.round(3))

The final distance between the ground truth and the predicted coordinates is : 1.045
The average error between the ground truth and the predicted coordinates is : 0.573


In [14]:
output_coord_3=np.zeros([11,inputs_test_3.shape[1],2])
for j in range(11):
    for i in range(inputs_test_3.shape[1]):
        if j==0:
            output_coord_3[j,i,0:2] = in_test_coord_3[9,i,0:2]
        else:
            output_coord_3[j,i,0:2] = output_test_3[j-1,i,:2]*0.4 + output_coord_3[j-1,i,0:2]
            

dist = np.zeros(inputs_test_3.shape[1])
for i in range(inputs_test_3.shape[1]):
    dist[i] = np.sqrt(sum((gt_test_coord_3[10,i,:]-output_coord_3[10,i,:])**2))

final_coord_error = np.mean(dist)
print('The final distance between the ground truth and the predicted coordinates is :',
      final_coord_error.round(3))


avr = np.zeros((inputs_test_3.shape[1],11))
for i in range(inputs_test_3.shape[1]):
    for j in range(11):
        avr[i,j] = np.sqrt(sum((gt_test_coord_3[j,i,:]-output_coord_3[j,i,:])**2))

average = np.mean(np.mean(avr,1))
print('The average error between the ground trought and the predicted coordinates is :',
      average.round(3))

The final distance between the ground truth and the predicted coordinates is : 1.231
The average error between the ground trought and the predicted coordinates is : 0.651


In [15]:
output_coord=np.zeros([11,inputs_test.shape[1],2])
for j in range(11):
    for i in range(inputs_test.shape[1]):
        if j==0:
            output_coord[j,i,0:2] = in_test_coord[9,i,0:2]
        else:
            output_coord[j,i,0:2] = output_test[j-1,i,:2]*0.4 + output_coord[j-1,i,0:2]
        

dist = np.zeros(inputs_test.shape[1])
for i in range(inputs_test.shape[1]):
    dist[i] = np.sqrt(sum((gt_test_coord[10,i,:]-output_coord[10,i,:])**2))

final_coord_error = np.mean(dist)
print('The final distance between the ground truth and the predicted coordinates is :',
      final_coord_error.round(3))


avr = np.zeros((inputs_test.shape[1],11))
for i in range(inputs_test.shape[1]):
    for j in range(11):
        avr[i,j] = np.sqrt(sum((gt_test_coord[j,i,:]-output_coord[j,i,:])**2))

average = np.mean(np.mean(avr,1))
print('The average error between the ground trought and the predicted coordinates is :',
      average.round(3))

The final distance between the ground truth and the predicted coordinates is : 1.148
The average error between the ground trought and the predicted coordinates is : 0.616
